In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [4]:
def load_CIFAR10(num_class):
    """
    Load CIFAR10 dataset and data preprocessing.
    :param data_dir: Path where to find the data file.
    :return: Tuple of numpy arrays: (x_train, y_train), (x_test, y_test).
    """
    # Initialize Data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    y_train = tf.keras.utils.to_categorical(y_train, num_class)
    y_test = tf.keras.utils.to_categorical(y_test, num_class)
    
    return (x_train, y_train), (x_test, y_test)

(x, y), (x_test, y_test) = load_CIFAR10(10)

In [6]:
print(x.shape, y.shape)

(60000, 28, 28) (60000, 10)


In [5]:
def get_model():
    """Define and return the ResNet50V2 model.
    Args:
        input_shape: a tensor defined the data shape.
        num_classes: the number of classes.
    Returns: 
        a keras model object named ResNet50V2.
    """
    model = tf.keras.Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(512, activation=tf.nn.relu),
        Dropout(0.2),
        Dense(10, activation=tf.nn.softmax)
    ])
    model.summary()
    
    return model

model = get_model()

2022-05-07 18:24:51.777037: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-07 18:24:54.747333: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2022-05-07 18:24:54.748527: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 1 with properties: 
pciBusID: 0000:41:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2022-05-07 18:24:54.749682: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 2 with properties: 
pciBusID: 0000:45:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [60]:
initial_model = get_model()
for j in range(2):
    deltas = []
    for i in range(2):
        model = get_model()
        model.set_weights(initial_model.get_weights())
        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(lr=5e-5),
                      metrics=['accuracy'])
        model.fit(x[i*10000:i*10000+10000],
                  y[i*10000:i*10000+10000],
                  batch_size=128,
                  epochs=30,
                  verbose=1)
        delta = np.array(initial_model.get_weights()) - \
            np.array(model.get_weights())
        deltas.append(delta)
        if j==0:
            model.save('./weight/model_' + str(i) + '.h5')
    delt_av = (deltas[0] + deltas[1]) / 2
    new_weights = np.array(initial_model.get_weights()) - delt_av
    initial_model.set_weights(new_weights)

Model: "sequential_80"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_80 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_160 (Dense)            (None, 512)               401920    
_________________________________________________________________
dropout_80 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_161 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_81 (Flatten)         (None, 784)               0         
________________________

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


10000/10000 [==============================] - 1s 53us/sample - loss: 2.0093 - accuracy: 0.3934
Epoch 2/30
10000/10000 [==============================] - 0s 29us/sample - loss: 1.3693 - accuracy: 0.7279
Epoch 3/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.9824 - accuracy: 0.8052
Epoch 4/30
10000/10000 [==============================] - 0s 35us/sample - loss: 0.7676 - accuracy: 0.8364
Epoch 5/30
10000/10000 [==============================] - 0s 33us/sample - loss: 0.6397 - accuracy: 0.8587
Epoch 6/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.5638 - accuracy: 0.8659
Epoch 7/30
10000/10000 [==============================] - 0s 30us/sample - loss: 0.5090 - accuracy: 0.8757
Epoch 8/30
10000/10000 [==============================] - 0s 30us/sample - loss: 0.4658 - accuracy: 0.8894
Epoch 9/30
10000/10000 [==============================] - 0s 30us/sample - loss: 0.4399 - accuracy: 0.8914
Epoch 10/30
10000/10000 [=======================

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or 

10000/10000 [==============================] - 1s 53us/sample - loss: 0.2347 - accuracy: 0.9360
Epoch 2/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2272 - accuracy: 0.9374
Epoch 3/30
10000/10000 [==============================] - 0s 33us/sample - loss: 0.2244 - accuracy: 0.9390
Epoch 4/30
10000/10000 [==============================] - 0s 36us/sample - loss: 0.2154 - accuracy: 0.9424
Epoch 5/30
10000/10000 [==============================] - 0s 39us/sample - loss: 0.2111 - accuracy: 0.9414
Epoch 6/30
10000/10000 [==============================] - 0s 39us/sample - loss: 0.2067 - accuracy: 0.9422
Epoch 7/30
10000/10000 [==============================] - 0s 36us/sample - loss: 0.2020 - accuracy: 0.9454
Epoch 8/30
10000/10000 [==============================] - 0s 39us/sample - loss: 0.1982 - accuracy: 0.9464
Epoch 9/30
10000/10000 [==============================] - 0s 44us/sample - loss: 0.1938 - accuracy: 0.9471
Epoch 10/30
10000/10000 [=======================

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


10000/10000 [==============================] - 1s 55us/sample - loss: 0.2459 - accuracy: 0.9318
Epoch 2/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2394 - accuracy: 0.9322
Epoch 3/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2326 - accuracy: 0.9358
Epoch 4/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2268 - accuracy: 0.9370
Epoch 5/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2239 - accuracy: 0.9380
Epoch 6/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2148 - accuracy: 0.9429
Epoch 7/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2113 - accuracy: 0.9427
Epoch 8/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2040 - accuracy: 0.9464
Epoch 9/30
10000/10000 [==============================] - 0s 29us/sample - loss: 0.2004 - accuracy: 0.9475
Epoch 10/30
10000/10000 [=======================

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or 

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
target_model = keras.models.load_model('./weight/model_1.h5')
attack = RandomForestClassifier(n_estimators=50)
y_train = np.zeros(60000)
for i in range(10000):
    y_train[i] = 1
x_train = np.vstack((x[10000:20000], x_test))
attack.fit(model.predict_proba(x_train), y_train[0:20000])
xt = np.vstack((x[0:10000], x_test))
yt = y_train[0:20000]
y_pred = attack.predict(initial_model.predict_proba(xt))
print(classification_report(y_true=yt, y_pred=y_pred))

              precision    recall  f1-score   support

         0.0       0.62      0.81      0.70     10000
         1.0       0.72      0.50      0.59     10000

    accuracy                           0.65     20000
   macro avg       0.67      0.65      0.65     20000
weighted avg       0.67      0.65      0.65     20000



In [19]:
xt.shape

(20000, 28, 28)

In [41]:
print(attack.predict(initial_model.predict_proba(x)))

[1. 1. 1. ... 1. 1. 1.]
